# Read and clean a single file

In [22]:
import os
import sys
import logging
import pandas as pd

from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'yuma_bucket'
directory = 'BigData'

## Read Metadata
-----
General info for each relevant column

In [23]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


The final data must contains this and only this columns with that specific headers in order to standarize the data, this is a type of data dictionary but it also must include the data types (**schema**) of each field (**column**)

## Get raw data

In [7]:
raw_data_oct = get_input_data(bucket=bucket,
                          initial_directory=directory,
                          filename = 'datos-abiertos-octubre-2019.csv')
print(raw_data_oct.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18195 entries, 0 to 18194
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NUMERO_INCIDENTE     18195 non-null  int64 
 1   FECHA_DESPACHO_518   18195 non-null  object
 2   CODIGO DE LOCALIDAD  18195 non-null  int64 
 3   LOCALIDAD            18195 non-null  object
 4   EDAD                 18195 non-null  object
 5   UNIDAD               9503 non-null   object
 6   GENERO               9543 non-null   object
 7   RED                  18195 non-null  object
 8   TIPO_INCIDENTE       18195 non-null  object
 9   PRIORIDAD            18195 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.4+ MB
None


It seems that this file contains already the fields, but need to rename them

### Rename Columns

In [24]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data_oct.rename(
    columns = {
        'NUMERO_INCIDENTE'                  : 'FECHA_INCIDENTE',
        'FECHA_DESPACHO_518'                : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
    },
    inplace=True
)
raw_data_oct.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA


### Check values per field
--------
1. FECHA_INICIO_DESPLAZAMIENTO_MOVIL
2. LOCALIDAD
3. GENERO
4. LOCALIDAD            
5. EDAD               
6. UNIDAD            
7. GENERO              
8. RED                  
9. TIPO_INCIDENTE      
10. PRIORIDAD

In [25]:
col = 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL'
raw_data_oct[col] = pd.to_datetime(raw_data_oct[col], format='%Y/%m/%d %H:%M') 
raw_data_oct[col].value_counts
raw_data_oct.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA


In [26]:
col = 'LOCALIDAD'
raw_data_oct[col] = raw_data_oct[col].apply(lambda x: x.strip())
raw_data_oct[col].value_counts(dropna=False)

Kennedy               2311
Engativa              1617
Suba                  1572
Bosa                  1220
Ciudad Bolivar        1062
Puente Aranda          889
Rafael Uribe Uribe     872
Usaquen                806
Fontibon               793
San Cristobal          782
Chapinero              682
Santa Fe               631
Los Martires           628
Barrios Unidos         619
Usme                   557
Tunjuelito             533
Teusaquillo            484
Antonio Nariño         384
La Candelaria          112
Sumapaz                  5
Name: LOCALIDAD, dtype: int64

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [27]:
col = 'GENERO'
#raw_data_oct[col] = raw_data_oct[col].apply(lambda x: x.strip())
raw_data_oct[col].value_counts(dropna=False)

NaN          7064
MASCULINO    5118
FEMENINO     4354
SIN_DATO       23
Name: GENERO, dtype: int64

In [15]:
raw_data_oct[col].fillna('SIN_DATO').value_counts()

SIN_DATO     8680
MASCULINO    5145
FEMENINO     4370
Name: GENERO, dtype: int64

In [28]:
col = 'RED'
raw_data_oct[col] = raw_data_oct[col].apply(lambda x: x.strip())
raw_data_oct[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

Norte      7262
Sur        6019
Privada    3278
Name: RED, dtype: int64

In [29]:
col = 'TIPO_INCIDENTE'
raw_data_oct[col] = raw_data_oct[col].apply(lambda x: x.strip())
raw_data_oct[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    6015
Inconsciente/Paro Cardiorrespiratorio        1622
Dificultad Respiratoria                      1420
Enfermo                                      1194
Convulsiones                                 1035
Heridos                                       772
Dolor torácico                                675
Trastorno mental                              627
Intento de suicidio                           594
Lesiones personales                           560
Accidente cerebro vascular                    374
Patología Gineco - obstétrica                 362
Síntomas gastrointestinales                   359
Caída                                         300
Intoxicaciones                                159
Ideas de suicidio                             101
SIN_DATO                                       54
Incendio estructural                           41
Muerte Natural                                 33
Solicitud Apoyo / Desacato                     30


In [30]:
col = 'PRIORIDAD'
raw_data_oct[col] = raw_data_oct[col].apply(lambda x: x.strip())
raw_data_oct[col].value_counts(dropna=False)

ALTA        13754
MEDIA        2578
BAJA          189
SIN_DATO       38
Name: PRIORIDAD, dtype: int64

# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [23]:
project_dir

'/home/jupyter/ESEIT_BigData'

In [31]:
profile = ProfileReport(df=raw_data_oct)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_octubre_2019.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Remove duplicates

In [32]:
print('Number of rows before cleaning:', raw_data_oct.shape[0] )
raw_data_oct.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data_oct.shape[0] )

Number of rows before cleaning: 16559
Number of rows after cleaning: 16559


### Save the final table
use to_csv to save the table

In [33]:
# saving in a local directory
raw_data_oct.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_octubre_2019.csv', encoding='latin1', sep=';', index=False)